In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import os
import json

RESULTS_PATH = ''

def msd_id_to_dirs(msd_id):
    """Given an MSD ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return os.path.join(msd_id[2], msd_id[3], msd_id[4], msd_id)

def get_midi_path(msd_id, midi_md5, kind):
    """Given an MSD ID and MIDI MD5, return path to a MIDI file.
    kind should be one of 'matched' or 'aligned'. """
    return os.path.join(RESULTS_PATH, 'lmd_{}'.format(kind),
                        msd_id_to_dirs(msd_id), midi_md5 + '.mid')

def sortAndPP(dicte):
    print(json.dumps({key: value for key, value in sorted(dicte.items(), key=lambda item: item[1])}, indent=4))

# Genre technique 1 (finer grained genres but less content)  (not used)
Clean midi subset used instead for now as [github content](https://gist.github.com/naotokui/3a441962a0d427c6c3a49431c34a1666) available for genre information, scraped from gracenote api)

Though this finishes prematurely.

In [4]:
df = json.load(open('lakh_clean_midi_genres_edit.json','r'))
print("lakh clean len", len(df))
genres = {}
for song in df:
    if 'genres' in song:
        for genre in song['genres']:
            if genre in genres:
                genres[genre] += 1
            else:
                genres[genre] = 1
    else:
        pass
        #print(song)
    
print(len(genres))
#print(genres)
sortAndPP(genres)

lakh clean len 5864
307
{
    "Audio Book": 1,
    "Contemporary Instrumental": 1,
    "Japanese Pop": 1,
    "J-Pop": 1,
    "Progressive Country": 1,
    "Southern Rap": 1,
    "Two-Tone Ska Revival": 1,
    "Central American": 1,
    "Belizean": 1,
    "Jump Blues": 1,
    "Thrash Metal": 1,
    "Breakbeat": 1,
    "Pop Electronica": 1,
    "Texas Blues": 1,
    "Progressive Bluegrass": 1,
    "Surf Rock": 1,
    "General Spoken Word": 1,
    "Exotica": 1,
    "Latin House": 1,
    "Noise Rock": 1,
    "Western Swing": 1,
    "Classic Pop Punk": 1,
    "Jangle Pop": 1,
    "Deep House": 1,
    "Neo-Glam": 1,
    "Musical Film/TV Music": 1,
    "General Punk": 1,
    "Avant Garde": 1,
    "Rock Singer-Songwriter": 1,
    "Gypsy Jazz": 1,
    "Progressive House": 1,
    "General Classical Crossover": 1,
    "Psychedelic Pop": 1,
    "Latin Folk & Traditional": 1,
    "Original Post-Punk": 1,
    "Bebop": 1,
    "Rock Opera": 1,
    "Pop Choral": 1,
    "General New Age": 1,
    "Moder

# Genre technique 2

LMD-matched to 1 million songs database (a subset of lmd full with correlated tracks)]

LMD-matched HDF5 metadata which contains general song information but no genre specific information.

Use external [All Music Guide](http://www.ifs.tuwien.ac.at/mir/msd/) information to get genre information.
(http://www.ifs.tuwien.ac.at/mir/msd/partitions/msd-MAGD-genreAssignment.cls) This was created for msd so not necesarilly files found in lakhMidi

In [5]:
import pandas as pd
msdGenres = pd.read_csv('msd-MAGD-genreAssignment.cls',names=['msd','genre'],sep="\t")


with open('match_scores.json') as f:
    msdToLakh = json.load(f)
# scores['TRNCZVX128F92F9018']
msdGenres.shape[0]

422714

In [6]:
from tqdm.notebook import tqdm as log_progress

hits = 0
for i, row in log_progress(msdGenres.iterrows()):
    if row[0] in msdToLakh.keys():
         hits += 1

In [7]:
print(f"Hits: {hits}, {(100*hits/msdGenres.shape[0]):.2f}% of genre'd msds matched, {(100*hits/45129):.2f}% of lakh matched")
boolCol = msdGenres.isin(msdToLakh.keys()).loc[:,'msd']

lakhGenres = msdGenres.where(boolCol).dropna()
lakhGenres['genre'].value_counts()

Hits: 10788, 2.55% of genre'd msds matched, 23.90% of lakh matched


Pop_Rock          6181
Electronic        1171
Country            711
RnB                520
Latin              492
International      304
Jazz               250
Religious          211
Rap                204
Vocal              181
Easy_Listening     140
New Age            111
Folk                68
Reggae              51
Classical           42
Stage               40
Children            39
Blues               38
Comedy_Spoken       17
Holiday             11
Avant_Garde          6
Name: genre, dtype: int64

In [16]:
def msdToDirs(msd):
    #assumes the msd is one that is matches
    # could modify to only return one .mid for each msd with highest confidence
    return [get_midi_path(msd, md5, 'matched') for md5 in msdToLakh[msd].keys()]
    
def genreToDirs(genre):
    out = []
    for msd in genreGrouped.get_group(genre)['msd']:
        out.extend(msdToDirs(msd))
    return out

genreGrouped = lakhGenres.groupby('genre')
#genreGrouped.groups

2996      TRAERLD128F425A9A8
3583      TRAFOZW128F92F1594
4799      TRAHMHQ12903CEC9A9
6053      TRAJMBU128F149332A
7349      TRALNTD128F4227418
                 ...        
411853    TRZINCP128F4233625
414627    TRZNAMF128F931A9D4
417190    TRZRDTW128E0793A97
417718    TRZRZUO12903CAF163
421059    TRZXKYZ128E07854D2
Name: msd, Length: 181, dtype: object

In [9]:
genreMidiAmounts = {}
for genre in genreGrouped.groups:
    genreMidiAmounts[genre] = len(genreToDirs(genre))
    
sortAndPP(genreMidiAmounts)

{
    "Avant_Garde": 24,
    "Holiday": 47,
    "Comedy_Spoken": 92,
    "Blues": 100,
    "Stage ": 105,
    "Reggae": 141,
    "Folk": 200,
    "Children": 282,
    "Classical": 285,
    "New Age": 496,
    "Religious": 606,
    "Vocal": 698,
    "Rap": 701,
    "Easy_Listening": 929,
    "International": 1008,
    "Jazz": 1179,
    "Latin": 1410,
    "RnB": 2040,
    "Country": 2410,
    "Electronic": 3460,
    "Pop_Rock": 21024
}


In [24]:
def msdToDirsMostConfidence(msd):
    # returns md5 of most confidence
    return [get_midi_path(msd, max(msdToLakh[msd], key=msdToLakh[msd].get), 'matched')]
    
def genreToDirsLess(genre):
    out = []
    for msd in genreGrouped.get_group(genre)['msd']:
        out.extend(msdToDirsMostConfidence(msd))
    return out


from os.path import basename
import zipfile

lista_files = genreToDirsLess('Pop_Rock')
with zipfile.ZipFile('out.zip', 'w') as zipMe:        
    for file in log_progress(lista_files):
        zipMe.write(file,basename(file), compress_type=zipfile.ZIP_DEFLATED)
ZipFile.close()

/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '020ae8f827b69c9e6f953284f75542c6.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '21d2123e7692d2bdfde6bf4bcba692db.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '9b6daa1bd54bb76bb396274ef6d467fa.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'fc9c5aab1a09c0dc5e8ed605546e743f.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '9a0208c5eefa89ef1d05a8a2a06acdc8.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/minicond

/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'cf243a8a8e4864baef8b18db61ba0d04.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '9bd67d134297a0042edb4cd3ff3681a8.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '1a6365965d275307952e5d9ed949409e.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '811728248e1265e8e296b719700f1a10.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'd9544d7f3a4642a3e096c97aa7d04603.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/minicond

/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '9fecf8b208f0630385dafe0e14ab824d.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '7ed436e32ead3df8caff96ade7fead47.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'ac5fa89c7f8ac1490a69dc1d8e3e66e9.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'cdb577e3649fbec42367d0dc1f4d4003.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '85edbb41010a8d1de3075262952eca38.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/minicond

/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'fa0b2f3f379bde58029a199dc14aa321.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '520ab7ccc4d161d6eefd69fb0affca17.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'b4680293c3b8c8cd4ba280df85b69194.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '9ceaf57be69ee1b1633fd07e33ef17fc.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '8a13b760481235e1862fd6e1032c0618.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/minicond

/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '46054c557357aef34614e153f75a7c27.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '5a70b9b417abe327bb3e6da697536fd4.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'ec9eddf496ed798671456ed529959576.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '915a018d61f19a83a641250ff75b6366.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'cff36983f68880110b67e92b256b9823.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/minicond

...

/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '698392429f128e4954ce66017966b63e.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '3f52ef89ac6b613b5b020b57a9b162c7.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '30ea07982ca7a529a51d0e5152e9d113.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '5706a122284cb095cb9265aa8701cdeb.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'd3934d828d445e09dd910fb83735e6e6.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/minicond

/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '28ef521a47fc6b176a8bf2b6f427726a.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'e030e04ab75c60f7ee59fd9f6193052c.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '7dbf640b41e1dd1d0e94d0fa25f4fc9f.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'fb439b317ea488220ee5a0e3ce76303e.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'd0eafc1023fda33a88b423009061415d.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/minicond

/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '3c23ebb6de56319baba46e7ea39b65da.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '2fd70602ff727120d9b9e4b0c7bfb298.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'e602db5b6d946f616a1cee29fd52d69b.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'd8203b92622821eec0fa663686d76f9f.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '3967b33a00b2f3bcc49ba011e5a65865.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/minicond

/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: '1b2a4e15a13acffa1353fc8b2f28a3e9.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'ea1f89c4ad07debd124d978e6133f7f0.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'f331d343800918230ecac091540f0563.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'c710d7d1f1ecc502d2c706a434c24b22.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/miniconda3/envs/py3iaml/lib/python3.7/zipfile.py:1473: UserWarning: Duplicate name: 'e7ef9054ebba96ec3d0fce7dbf759cdb.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/Users/dima/minicond

NameError: name 'ZipFile' is not defined